In [4]:
%run packages.py

/Users/mcbookairdebat/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
from data_import_enginnering import DataImportEnginnering
from metric_selection import Pair_Selection
from data_visualization import Data_Visualization
import importlib

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# 1. DATA

## 1.1. Importation of data into a dictionary

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
Stock_list = pd.DataFrame(tables[0])
Stock_list.drop(['Security','GICS Sub-Industry','Headquarters Location','CIK','Founded'],axis = 1, inplace=True)
Stock_list['Date added'] = pd.to_datetime(Stock_list['Date added'])
Stock_list = Stock_list[( Stock_list['Date added'] < '2023-12-31') & (Stock_list['Date added']>'2015-01-01')][['Symbol','GICS Sector']]

In [7]:
stock_list = list(Stock_list['Symbol'].unique())
Stock_list_sector = Stock_list.groupby('GICS Sector')['Symbol'].unique().to_dict()
sector_list = list(Stock_list_sector.keys())
pairs = {k:[tuple(sorted((i,j))) for i in Stock_list_sector[k] for j in Stock_list_sector[k] if j!=i] for k in sector_list}
pairs = {k :sorted(list(set(pairs[k]))) for k in sector_list}

In [8]:
start_date = '2018-01-01'
end_date = '2022-12-31'
period = '1d'
colone = ['Close']


In [9]:
Enginner = DataImportEnginnering(stock_list,start_date=start_date,end_date=end_date,period=period,colone=colone)

In [10]:
raw_data = Enginner.RawData()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## 1.2. Normalisation of data

In [11]:
raw_data_normalized = {}
for key, value in raw_data.items():
    raw_data_normalized[key] = Enginner.normalize(value)

raw_data_normalized

{'AOS': Price          Close
 Ticker           AOS
 Date                
 2018-01-02  0.454586
 2018-01-03  0.461197
 2018-01-04  0.466378
 2018-01-05  0.480672
 2018-01-08  0.486032
 ...              ...
 2022-12-23  0.465824
 2022-12-27  0.480853
 2022-12-28  0.463287
 2022-12-29  0.480658
 2022-12-30  0.472460
 
 [1259 rows x 1 columns],
 'AMD': Price          Close
 Ticker           AMD
 Date                
 2018-01-02  0.009516
 2018-01-03  0.013256
 2018-01-04  0.016997
 2018-01-05  0.015422
 2018-01-08  0.018047
 ...              ...
 2022-12-23  0.360874
 2022-12-27  0.352671
 2022-12-28  0.348077
 2022-12-29  0.362843
 2022-12-30  0.362515
 
 [1259 rows x 1 columns],
 'ABNB': Price          Close
 Ticker          ABNB
 Date                
 2020-12-10  0.463119
 2020-12-11  0.422479
 2020-12-14  0.353629
 2020-12-15  0.314924
 2020-12-16  0.413100
 ...              ...
 2022-12-23  0.020543
 2022-12-27  0.007443
 2022-12-28  0.000000
 2022-12-29  0.020395
 2022-12-30  0.02240

In [12]:
dataframe_normalized = Enginner.Dictionnary_to_Dataframe(raw_data_normalized)
raw_dataframe = Enginner.Dictionnary_to_Dataframe(raw_data)


/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stock_name] = z[stock_name]
/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[stock_name] = z[stock_name]
/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning: Da

In [13]:
dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()
raw_dataframe= raw_dataframe.stack().stack().droplevel(level =2).unstack()

/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_75544/628406389.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()
/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_75544/628406389.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  raw_dataframe= raw_dataframe.stack().stack().droplevel(level =2).unstack()


## 1.3 Correlation Matrix

In [14]:
q = raw_dataframe.shape[1]/raw_dataframe.shape[0]
data = raw_dataframe.copy()
data = np.log(data/data.shift(1))
data = data.apply(lambda x: x-x.mean())
data = data.iloc[1:,:]


In [15]:
correlation_matrix = data.corr()

# 2.Reduction Dimension with PCA and Marcenko Pastur 

## Marcenko pastur to keep only the rights one

In [16]:

def mpPDF(var,q):
    e_min,e_max = var* (1 -(1/q)**.5),var* (1 + (1/q)**.5)
    return e_min,e_max

from sklearn.neighbors._kde import KernelDensity
def getPCA(matrix):
    eVal,eVec = np.linalg.eigh(matrix)
    indices = eVal.argsort()[::-1]
    eVal,eVec = eVal[indices],eVec[:,indices]
    eVal = np.diagflat(eVal)
    return eVal,eVec

def fitKDE(obs,bWidth =.25,kernel = 'gaussian', x=None):
    if len(obs.shape)==1: obs =obs.reshape(-1,1)
    kde = KernelDensity(bandwidth=bWidth,kernel=kernel).fit(obs)
    if x is None : x=np.unique(obs).reshape(-1,1)
    if len(x.shape)==1 : x=x.reshape(-1,1)
    logProb = kde.score_samples(x)
    pdf = pd.Series(np.exp(logProb), index = x.flatten())
    return pdf

x = np.random.normal(size=(1000,1000))
eVal0,eVec0 = getPCA(correlation_matrix)
e_min,e_max = mpPDF(1.,raw_dataframe.shape[1]/raw_dataframe.shape[0])
mask = (np.diag(eVal0)>e_max)

In [17]:
mt =mask*np.diag(eVal0)
treshold = (mt>0).sum()
eVec0 = pd.DataFrame(eVec0).T


In [18]:
pca_proj = eVec0.iloc[:,:3]
new_corr = np.dot(correlation_matrix.T,pca_proj)
new_corr = pd.DataFrame(new_corr,index = correlation_matrix.index,columns =['X','Y','Z'])

# 3.Applying Optic/ DBScan and construction pair 

In [19]:
# Research 
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


X = StandardScaler().fit_transform(new_corr)
X =pd.DataFrame(X,index = new_corr.index, columns = ['X','Y','Z'])
X

,X,Y,Z
Ticker,,,
AOS,0.692635,0.181372,1.096769
AMD,-0.829708,1.576058,-1.068893
ALB,0.017931,0.607145,0.694085
ARE,-1.175985,0.356653,-0.990602
ALGN,1.001526,0.124225,-0.018979
...,...,...,...
CARR,0.412791,-1.199427,0.897410
OTIS,-0.485793,-1.087495,0.624770
ABNB,-0.642810,-1.074629,-0.325543


In [20]:
import numpy as np

from sklearn import metrics
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=3).fit(X)
labels = db.labels_
X['label'] =labels

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 14
Estimated number of noise points: 102


In [21]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(X, x='X', y='Y', z='Y',color='label')
fig.show()

In [22]:
X.reset_index(inplace=True, drop = False)


In [23]:
f_dic ={}
for tick in X.label.unique():
    if tick ==-1:
        pass
    else:
        f_dic[tick] = X[X['label']==tick]['Ticker']

In [24]:
pairs=[]
for val in f_dic.values():
    val1 = val.copy()
    for stock1 in val1:
        pairs.extend([(stock1,stock2)  for stock2 in val1 if stock2!=stock1])
        val1 = val1[1:]

# 4.Rules for pair selection 

## Check for cointegration

In [25]:
# Engle granger 
# first cointegration step 
Metric = Pair_Selection(dataframe_normalized,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list,sector_neutral = True)

In [29]:
selected_pair = Metric.paire_selection_2(pairs,p=0.05)
selected_pair[0][0]

'CDNS'

## Check that the spread is mean reverting

Ticker,CDNS,SNPS
Date,,
2018-01-02,0.039430,0.021818
2018-01-03,0.045027,0.025128
2018-01-04,0.049161,0.027795
2018-01-05,0.051768,0.031137
2018-01-08,0.057683,0.033804
...,...,...
2022-12-23,0.798779,0.774300
2022-12-27,0.789875,0.767038
2022-12-28,0.779000,0.758555


In [54]:
data1 = dataframe_normalized.copy()
data2 = raw_dataframe.copy()
# The goal would be to stock every small data frame in a dictionnay to help doing the steps for the reste 
dataframes_dictionnary_norm = {}
dataframes_dictionnary_raw ={}
for i in selected_pair: 
        dataframes_dictionnary_norm[((i[0]),(i[1]))] = data1[[(i[0]),(i[1])]].copy()
        dataframes_dictionnary_raw[((i[0]),(i[1]))] = data2[[(i[0]),(i[1])]].copy()
dataframes_dictionnary_norm

{('CDNS',
  'SNPS'): Ticker          CDNS      SNPS
 Date                          
 2018-01-02  0.039430  0.021818
 2018-01-03  0.045027  0.025128
 2018-01-04  0.049161  0.027795
 2018-01-05  0.051768  0.031137
 2018-01-08  0.057683  0.033804
 ...              ...       ...
 2022-12-23  0.798779  0.774300
 2022-12-27  0.789875  0.767038
 2022-12-28  0.779000  0.758555
 2022-12-29  0.801832  0.779988
 2022-12-30  0.793628  0.771344
 
 [1259 rows x 2 columns],
 ('MPWR',
  'SNPS'): Ticker          MPWR      SNPS
 Date                          
 2018-01-02  0.025389  0.021818
 2018-01-03  0.029840  0.025128
 2018-01-04  0.032076  0.027795
 2018-01-05  0.033032  0.031137
 2018-01-08  0.036060  0.033804
 ...              ...       ...
 2022-12-23  0.530994  0.774300
 2022-12-27  0.513364  0.767038
 2022-12-28  0.509132  0.758555
 2022-12-29  0.538593  0.779988
 2022-12-30  0.538996  0.771344
 
 [1259 rows x 2 columns],
 ('HOLX',
  'PODD'): Ticker          HOLX      PODD
 Date               

In [55]:


for key,values in dataframes_dictionnary_norm.items():
    values.loc[:,'Delta'] = values[key[0]] - values[key[1]]
    values.loc[:,'Delta_norm'] = (values['Delta'] - values['Delta'].mean()) / values['Delta'].std()
    values.drop(['Delta'], axis = 1, inplace=True )

dataframes_dictionnary_norm

('CDNS', 'SNPS')
('MPWR', 'SNPS')
('HOLX', 'PODD')


{('CDNS',
  'SNPS'): Ticker          CDNS      SNPS  Delta_norm
 Date                                      
 2018-01-02  0.039430  0.021818   -0.411659
 2018-01-03  0.045027  0.025128   -0.332429
 2018-01-04  0.049161  0.027795   -0.281611
 2018-01-05  0.051768  0.031137   -0.307051
 2018-01-08  0.057683  0.033804   -0.194541
 ...              ...       ...         ...
 2022-12-23  0.798779  0.774300   -0.173763
 2022-12-27  0.789875  0.767038   -0.230639
 2022-12-28  0.779000  0.758555   -0.313513
 2022-12-29  0.801832  0.779988   -0.265051
 2022-12-30  0.793628  0.771344   -0.249817
 
 [1259 rows x 3 columns],
 ('MPWR',
  'SNPS'): Ticker          MPWR      SNPS  Delta_norm
 Date                                      
 2018-01-02  0.025389  0.021818    0.622484
 2018-01-03  0.029840  0.025128    0.639874
 2018-01-04  0.032076  0.027795    0.633311
 2018-01-05  0.033032  0.031137    0.596971
 2018-01-08  0.036060  0.033804    0.602479
 ...              ...       ...         ...
 2022-12

In [63]:
# Hurst Exponent 
#Implementation of the R/S method 
val = dataframes_dictionnary_norm[('CDNS', 'SNPS')]
val['Delta_norm']

    

Date
2018-01-02   -0.411659
2018-01-03   -0.332429
2018-01-04   -0.281611
2018-01-05   -0.307051
2018-01-08   -0.194541
                ...   
2022-12-23   -0.173763
2022-12-27   -0.230639
2022-12-28   -0.313513
2022-12-29   -0.265051
2022-12-30   -0.249817
Name: Delta_norm, Length: 1259, dtype: float64

In [66]:
 from scipy import stats

def estimateHurstRS(X, min_s, max_s, step_s):
    # STEP 1: Initialize arrays to store log(R/S) and log(s)
    log_rs_values = []
    log_s_values  = []
    
    # Ensure the time series is in a usable numeric format
    N = len(X)
    
    # STEP 2: Iterate over different segment sizes
    for s in range(min_s, max_s + 1, step_s):

        # Check if s is valid in the context of the series length
        if s > 0 and s <= N:
            # We'll collect R/S values for all segments of size s
            rs_list = []
            
            # How many segments of size s can we fit?
            num_segments = int(np.floor(N / s))
            
            # STEP 2a: Loop over each segment
            for seg in range(0, num_segments):
                # Segment start and end indices
                start_idx = seg * s
                end_idx   = start_idx + s - 1
                
                # Extract the segment data
                segment_data = X[start_idx : end_idx+1]
                
                # STEP 2b: Compute the mean of the segment
                seg_mean = np.mean(segment_data)
                
                # STEP 2c: Compute the cumulative deviations 
                #   Y[k] = sum(X[i] - seg_mean) for i up to k in the segment
                Y = np.zeros(s)
                cumulative_sum = 0
                for k in range(0, s):
                    cumulative_sum += (segment_data[k] - seg_mean)
                    Y[k] = cumulative_sum
                
                # STEP 2d: Compute the range R = max(Y) - min(Y)
                R = max(Y) - min(Y)
                
                # STEP 2e: Compute the standard deviation S of the segment
                std_dev = np.std(segment_data)
                
                # STEP 2f: Compute R/S, handle division by zero if necessary
                if std_dev != 0:
                    rs_value = R / std_dev
                else:
                    rs_value = 0   # or skip if std_dev = 0
                
                # Collect R/S value for this segment
                rs_list.append(rs_value)
            
            # STEP 2g: Compute the average R/S over all segments of size s
            if len(rs_list) > 0:
                avg_rs = np.mean(rs_list)
                
                # Store the log of avg R/S and log of segment size s
                log_rs_values.append(np.log(avg_rs))
                log_s_values.append(np.log(s))
    
    # STEP 3: Fit a linear regression to log(R/S) vs. log(s)
    #   log(R/S) = H * log(s) + c
    # The slope of this regression is our estimate of H.
    H = stats.linregress(log_s_values, log_rs_values)
    
    return H
estimateHurstRS(val['Delta_norm'],5,200,5)

/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_75544/2258241596.py:39: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



LinregressResult(slope=np.float64(0.9658025651309837), intercept=np.float64(-0.8130680651584528), rvalue=np.float64(0.9984393741538202), pvalue=np.float64(3.1306997391283393e-49), stderr=np.float64(0.00876334164544645), intercept_stderr=np.float64(0.03901252043432769))

## Check that the spread is tradable 

In [52]:
#

## Check spread reversion is consistent 


In [53]:
# mean_cross superior to 12 